In [3]:
import sys
sys.path.append("..")

In [4]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [5]:
device = 'cpu'
np.random.seed(0)

dataset = IncomeDataset(device=device, sensitive_feature_labels=["SEX", "AGEP"])

In [8]:
def pareto_frontier_multi(myArray):
    # Sort on first dimension
    myArray = myArray[myArray[:,0].argsort()[::-1]]
    # Add first row to pareto_frontier
    pareto_frontier = myArray[0:1,:]
    # Test next row against the last row in pareto_frontier
    for row in myArray[1:,:]:
        if row[0]<pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier = np.concatenate((pareto_frontier, [row]))
        if row[0]==pareto_frontier[-1][0] and row[1]<pareto_frontier[-1][1]:
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier[-1] = row
    return pareto_frontier

def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def kde_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_kde_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [11]:
EI_hp_test = {}
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024
EI_hp_test['fairness'] = ''
EI_hp_test['h'] = 0.01
EI_hp_test['delta_huber'] = 0.5
EI_hp_test['delta_effort'] = 3.1
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0,0.2,0.4,0.6,0.8,0.9]
EI_hp_test['fairness'] = 'EI'
seeds = np.arange(3)
_, results_kde = kde_tradeoff(dataset, EI_hp_test, seeds)
results_kde_pareto = pareto_frontier_multi(np.squeeze(results_kde))

training seed 0 started


Training: 100%|██████████| 100/100 [05:03<00:00,  3.04s/epochs]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [04:58<00:00,  2.99s/epochs]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [04:55<00:00,  2.95s/epochs]


Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [04:54<00:00,  2.95s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  ----------
          0.001        0            0.817744        0.818315  0.0316439   0.0774129  0.0544926   0.0544926
          0.001        0            0.818207        0.814802  0.026737    0.0772871  0.0565781   0.0565781
          0.001        0            0.816746        0.819944  0.0387139   0.0765501  0.0481311   0.0481311
          0.001        0.2          0.817409        0.817613  0.0248719   0.0741094  0.0497085   0.0497085
          0.001        0.2          0.818103        0.815185  0.0222929   0.0733903  0.0522642   0.0522642
          0.001        0.2          0.816578        0.82004   0.0305259   0.0733188  0.0440053   0.0440053
          0.001        0.4          0.817137        0.817549  0.019922    0.0722119  0.0472375   0.0472375
  

In [13]:
np.save('results/income_tradeoff_kde_ei.npy',results_kde_pareto)

In [14]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def fb_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fb_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [15]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0] 
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024
EI_hp_test['delta_effort'] = 3.1
EI_hp_test['lambda_'] = [0,0.1,0.2,0.4,0.6,0.8]
EI_hp_test['fairness'] = 'EI'

_, results_fb = fb_tradeoff(dataset, EI_hp_test, seeds)
results_fb_pareto = pareto_frontier_multi(np.squeeze(results_fb))

training seed 0 started


Training: 100%|██████████| 100/100 [05:00<00:00,  3.01s/epochs]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [05:09<00:00,  3.10s/epochs]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [04:45<00:00,  2.85s/epochs]


Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [04:48<00:00,  2.89s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo        eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  ----------
          0.001        0            0.817744        0.818315  0.0316439   0.0774129  0.0544926   0.0544926
          0.001        0            0.818207        0.814802  0.026737    0.0772871  0.0565781   0.0565781
          0.001        0            0.816746        0.819944  0.0387139   0.0765501  0.0481311   0.0481311
          0.001        0.1          0.817065        0.81691   0.00134313  0.054236   0.0217971   0.0217971
          0.001        0.1          0.816993        0.814866  0.00145689  0.0507419  0.0208343   0.0208343
          0.001        0.1          0.814989        0.819561  0.00592885  0.0523131  0.0160258   0.0160258
          0.001        0.2          0.816211        0.815664  0.00663289  0.055351   0.0243164   0.0243164
  

In [16]:
np.save('results/income_tradeoff_fb_ei.npy',results_fb_pareto)

In [21]:
def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(int(seeds[i]))
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def fc_tradeoff(dataset, hp_test, seeds):
    hp = hp_test.copy()
    result = []
    result_test = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            for seed in seeds:
                c = []
                c_test = []
                hp['learning_rate'] = i
                hp['lambda_'] = k
                train, val, test = lr_fc_model_runner(dataset, hp, seeds=[seed])
                c.append(hp['learning_rate'])
                c.append(hp['lambda_'])
                c.append(train['accuracy_mean'])
                c.append(val['accuracy_mean'])
                c.append(val['ei_mean'])
                c.append(val['dp_mean'])
                c.append(val['eo_mean'])
                c.append(val['eodd_mean'])
                c_test.append(test['accuracy_mean'])
                c_test.append(test['ei_mean'])
                result_test.append(c_test)
                result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))
    return result, result_test

In [22]:
EI_hp_test = {}
EI_hp_test['learning_rate'] = [0.001]
EI_hp_test['lambda_'] = [0]
EI_hp_test['n_epochs'] = 100
EI_hp_test['batch_size'] = 1024
EI_hp_test['delta_effort'] = 3.1
EI_hp_test['lambda_'] = [0,0.5,0.75,0.85,0.9,0.95]
EI_hp_test['fairness'] = 'EI'
seeds = np.arange(3)
_, results_fc = fc_tradeoff(dataset, EI_hp_test, seeds)

training seed 0 started


Training: 100%|██████████| 100/100 [04:49<00:00,  2.90s/epochs]


Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [04:57<00:00,  2.97s/epochs]


Training finished for all seeds.
training seed 2 started


...

Training finished for all seeds.
training seed 1 started


Training: 100%|██████████| 100/100 [04:44<00:00,  2.85s/epochs]


Training finished for all seeds.
training seed 2 started


Training: 100%|██████████| 100/100 [04:44<00:00,  2.85s/epochs]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
          0.001       0             0.817744        0.818315  0.0316439   0.0774129  0.0544926  0.0544926
          0.001       0             0.818207        0.814802  0.026737    0.0772871  0.0565781  0.0565781
          0.001       0             0.816746        0.819944  0.0387139   0.0765501  0.0481311  0.0481311
          0.001       0.5           0.815971        0.816942  0.0258656   0.0783484  0.0556239  0.0556239
          0.001       0.5           0.81681         0.814131  0.0228834   0.0763195  0.0564573  0.0564573
          0.001       0.5           0.815348        0.818858  0.0338167   0.0769601  0.0491023  0.0491023
          0.001       0.75          0.813735        0.813939  0.0180518   0.078285   0.0556411  0.0556411
          0.0

In [24]:
results_fc_pareto = pareto_frontier_multi(np.squeeze(results_fc))

In [25]:
results_fc_pareto

array([[0.81933407, 0.02910575],
       [0.81869522, 0.02782595],
       [0.81823525, 0.02324149],
       [0.8175964 , 0.02300928],
       [0.816932  , 0.02265909],
       [0.81516878, 0.01558232],
       [0.81194899, 0.01242045],
       [0.81159124, 0.01150105],
       [0.80921473, 0.00889614],
       [0.80226407, 0.00680697],
       [0.80200853, 0.00584074]])

In [26]:
np.save('results/income_tradeoff_fc_ei.npy',results_fc_pareto)